In [1]:
import pandas as pd
import os
from binaryninja import *
import jupyter_black

jupyter_black.load()

os.environ["BN_DISABLE_USER_SETTINGS"] = "True"
os.environ["BN_DISABLE_USER_PLUGINS"] = "True"
os.environ["BN_DISABLE_REPOSITORY_PLUGINS"] = "True"

<h1>Static Single Assignment (SSA) with Binary Ninja</h1>

- Every variable is defined only once.
    - If a change is made a new version is created.
    - Using SSA we can see the variables source and determine the definition, and use, which is called the def-use chain.
    - If a variable is derived from
        - another node,
        - or has taken multiple paths,
        - or is wrapped in another function
        - we can use phi ϕ with the SSA form to determine the possible values used to create it, or keep walking back until we do.
        - a phi ϕ example in Binary Ninja:
            - r2_1#4 = ϕ(r2_1#1, r2_1#2)<br></br>
- Compiler optimisation enabled through the use of SSA:
    - Constant propagation – conversion of computations from runtime to compile time, e.g. treat the instruction a=3*4+5; as if it were a=17;
    - Value range propagation – precompute the potential ranges a calculation could be, allowing for the creation of branch predictions in advance.
    - Sparse conditional constant propagation – range-check some values, allowing tests to predict the most likely branch.
    - Dead-code elimination – remove code that will have no effect on the results.
    - Global value numbering – replace duplicate calculations producing the same result.
    - Partial-redundancy elimination – removing duplicate calculations previously performed in some branches of the program.

<h1>Load the binary view</h1>

Get the Trivision webs binary:
- wget https://github.com/therealsaumil/emux/raw/master/files/emux/TRI227WF/rootfs.tar.bz2
- bzip2 -d ./rootfs.tar.bz2
- tar -xvf ./rootfs.tar
- mkdir ./Tenda/BNDB/
- cp ./rootfs/usr/bin/webs ./Tenda/BNDB/TRI227WF_webs
- rename the binary as needed.
- open the binary in Binary Ninja and save a bndb.

In [2]:
bv = binaryninja.load("./Tenda/BNDB/trivision_webs.bndb")

<h1>Get the MLIL SSA form for function 0xB7E8</h1>

In [3]:
func = bv.get_function_at(0xB7E8)

mlil_ssa = func.mlil.ssa_form

for block in mlil_ssa:
    for insn in block:
        print(f"{insn.instr_index}  @  {hex(insn.address)}  {insn}")

0  @  0xb7ec  r6#1 = arg1#0
1  @  0xb7f0  r5#1 = arg2#0
2  @  0xb7f4  result#1, mem#1 = 0x9710(arg1#0) @ mem#0
3  @  0xb7f8  result_1#1 = result#1
4  @  0xb7fc  r3#1 = r6#1 + result#1
5  @  0xb800  goto 6 @ 0xb818
6  @  0xb818  r2_1#1 = ϕ(r2#0, r2_1#2)
7  @  0xb818  r3#2 = ϕ(r3#1, r3#3)
8  @  0xb818  result_1#2 = ϕ(result_1#1, result_1#4)
9  @  0xb818  if (result_1#2 s>= 0) then 10 @ 0xb804 else 13 @ 0xb81c
10  @  0xb804  r2_1#2 = zx.d([r3#2].b @ mem#1)
11  @  0xb804  r3#3 = r3#2 - 1
12  @  0xb80c  if (r2_1#2 == 0x2e) then 14 @ 0xb824 else 15 @ 0xb810
13  @  0xb81c  goto 17 @ 0xb84c
14  @  0xb824  if (result_1#2 == 0) then 21 else 22 @ 0xb828
15  @  0xb810  result_1#4 = result_1#2 - 1
16  @  0xb810  goto 6 @ 0xb818
17  @  0xb84c  r2_1#4 = ϕ(r2_1#1, r2_1#2)
18  @  0xb84c  r3#4 = ϕ(r3#2, r3#3)
19  @  0xb84c  [r5#1].b = 0 @ mem#1 -> mem#4
20  @  0xb84c  goto 29 @ 0xb854
21  @  0xb824  goto 17 @ 0xb84c
22  @  0xb828  r2_2#3 = result_1#2 + 1
23  @  0xb82c  r1#1 = r6#1
24  @  0xb830  r0#2 = 

<h1>Check the def-use for the 3rd param to memcpy</h1> 

In [4]:
mlil_ssa[25]

<MediumLevelILCallSsa: result#3, mem#2 = 0x93e0(r0#2, r1#1, r2_2#3) @ mem#1>

In [5]:
mlil_ssa[25].params[2].src

<SSAVariable: r2_2 version 3>

In [6]:
mlil_ssa[25].params[2].src.def_site

<MediumLevelILSetVarSsa: r2_2#3 = result_1#2 + 1>

In [7]:
mlil_ssa[25].params[2].src.use_sites

[<MediumLevelILCallSsa: result#3, mem#2 = 0x93e0(r0#2, r1#1, r2_2#3) @ mem#1>,
 <MediumLevelILVarPhi: r2_1#5 = ϕ(r2_2#3, r2_1#4)>]

<h1>Check if the 3rd param to memcpy takes negative range values</h1>

In [8]:
mlil_ssa[25].params[2].possible_values.ranges[0]

<range: -0x80000000 to -0x80000000>

In [9]:
mlil_ssa[25].params[2].possible_values.ranges[0].end

-2147483648

<h1>Check the def-use for phi nodes

In [10]:
ssa = mlil_ssa[17]

In [11]:
ssa

<MediumLevelILVarPhi: r2_1#4 = ϕ(r2_1#1, r2_1#2)>

In [12]:
(mlil_ssa.get_ssa_var_definition((ssa).src[0]))

<MediumLevelILVarPhi: r2_1#1 = ϕ(r2#0, r2_1#2)>

In [13]:
(mlil_ssa.get_ssa_var_uses((ssa).src[0]))

[<MediumLevelILVarPhi: r2_1#4 = ϕ(r2_1#1, r2_1#2)>]

In [14]:
func = bv.get_function_at(0xB7E8)
mlil_ssa = func.mlil.ssa_form

data = []

for block in mlil_ssa:
    for insn in block:
        instr_index = insn.instr_index
        address = hex(insn.address)
        instruction = str(insn)

        definitions = []
        if hasattr(insn, "src") and isinstance(insn.src, list):
            for operand in insn.src:
                if hasattr(operand, "version"):
                    def_insn = mlil_ssa.get_ssa_var_definition(operand)
                    if def_insn:
                        definitions.append(
                            f"{operand} @ {hex(def_insn.address)} [{def_insn.instr_index}]"
                        )

        uses = []
        if hasattr(insn, "dest") and hasattr(insn.dest, "version"):
            use_list = mlil_ssa.get_ssa_var_uses(insn.dest)
            for use in use_list:
                uses.append(f"{hex(use.address)} [{use.instr_index}]")

        data.append(
            [
                instr_index,
                address,
                instruction,
                "\n".join(definitions) if definitions else "None",
                "\n".join(uses) if uses else "None",
            ]
        )

headers = ["Instr Index", "Address", "Instruction", "Definitions", "Uses"]
df = pd.DataFrame(data, columns=headers)

# Write to CSV without index
df.to_csv("mlil_ssa_output.csv", index=False)

# Write to Parquet without index
# df.to_parquet("mlil_ssa_output.parquet", index=False)

In [15]:
df

,Instr Index,Address,Instruction,Definitions,Uses
0,0,0xb7ec,r6#1 = arg1#0,None,0xb7fc [4]\n0xb82c [23]
1,1,0xb7f0,r5#1 = arg2#0,None,0xb84c [19]\n0xb830 [24]\n0xb838 [26]
2,2,0xb7f4,"result#1, mem#1 = 0x9710(arg1#0) @ mem#0",None,None
3,3,0xb7f8,result_1#1 = result#1,None,0xb818 [8]
4,4,0xb7fc,r3#1 = r6#1 + result#1,None,0xb818 [7]
5,5,0xb800,goto 6 @ 0xb818,None,None
6,6,0xb818,"r2_1#1 = ϕ(r2#0, r2_1#2)",<SSAVariable: r2_1 version 2> @ 0xb804 [10],0xb84c [17]
7,7,0xb818,"r3#2 = ϕ(r3#1, r3#3)",<SSAVariable: r3 version 1> @ 0xb7fc [4]\n<SSA...,0xb804 [10]\n0xb804 [11]\n0xb84c [18]
8,8,0xb818,"result_1#2 = ϕ(result_1#1, result_1#4)",<SSAVariable: result_1 version 1> @ 0xb7f8 [3]...,0xb818 [9]\n0xb824 [14]\n0xb810 [15]\n0xb828 [...
9,9,0xb818,if (result_1#2 s>= 0) then 10 @ 0xb804 else 13...,None,None


In [16]:
df["Definitions"] = df["Definitions"].astype(str)
df["Uses"] = df["Uses"].astype(str)

In [17]:
df[df["Uses"].str.contains("17", na=False)]

,Instr Index,Address,Instruction,Definitions,Uses
6,6,0xb818,"r2_1#1 = ϕ(r2#0, r2_1#2)",<SSAVariable: r2_1 version 2> @ 0xb804 [10],0xb84c [17]
10,10,0xb804,r2_1#2 = zx.d([r3#2].b @ mem#1),None,0xb818 [6]\n0xb80c [12]\n0xb84c [17]


In [18]:
df[df["Definitions"].str.contains("r2_1", na=False)]

,Instr Index,Address,Instruction,Definitions,Uses
6,6,0xb818,"r2_1#1 = ϕ(r2#0, r2_1#2)",<SSAVariable: r2_1 version 2> @ 0xb804 [10],0xb84c [17]
17,17,0xb84c,"r2_1#4 = ϕ(r2_1#1, r2_1#2)",<SSAVariable: r2_1 version 1> @ 0xb818 [6]\n<S...,0xb854 [31]
31,31,0xb854,"r2_1#5 = ϕ(r2_2#3, r2_1#4)",<SSAVariable: r2_2 version 3> @ 0xb828 [22]\n<...,None


<h1>Using the Binary Ninja GUI</h1>

Within Binary Ninja we can jump to 0xB7E8, and use current_mlil to do the following:
- Check if the 3rd param to memcpy takes negative range values
    - current_mlil.ssa_form[25].params[2].possible_values.ranges[0].end<br></br>
- Check the def-use for phi nodes
    - ssa = current_mlil.ssa_form[17]
    - (current_mlil.ssa_form.get_ssa_var_definition((ssa).src[0]))
    - (current_mlil.ssa_form.get_ssa_var_uses((ssa).src[0]))

<h2>References</h2>

- Binary Ninja Python API Reference: https://api.binary.ninja/
- Static Single Assignment Basics: https://docs.binary.ninja/dev/concepts.html#static-single-assignment-basics
- Static single-assignment form: https://en.wikipedia.org/wiki/Static_single-assignment_form
- Binary Ninja Intermediate Language Overview: https://docs.binary.ninja/dev/bnil-overview.html
- SSA Explained: https://carstein.github.io/binary%20ninja/2020/10/22/ssa-explained.html
- Vulnerability Modeling with Binary Ninja: https://blog.trailofbits.com/2018/04/04/vulnerability-modeling-with-binary-ninja/
- F'ing Around with Binary Ninja: 2.0 Release Party!: https://www.youtube.com/watch?v=hQ4qxMVbLcM
- Emulating the Tenda AC15 Router: https://armx.exploitlab.net/docs/emulating-tenda-ac15.html
- EMUX: https://github.com/therealsaumil/emux/